In [ ]:
from functools import partial
from pathlib import Path

import geopandas
import numpy
import pandas
import rasterio
from snail.intersection import get_cell_indices
from tqdm.auto import tqdm

tqdm.pandas()

In [ ]:
base_dir = Path("../results")

In [ ]:
damage_cost = geopandas.read_parquet(base_dir/"africa-latest_filter-road-tertiary/hazard-landslide-arup/damage_EAD_and_cost_per_trigger.geoparquet")

In [ ]:
damage_cost.to_file("africa-latest_damage_EAD_and_cost_per_trigger.gpkg")

In [ ]:
splits_dir = base_dir / 'direct_damages/africa-latest_filter-road-tertiary/hazard-landslide-arup/split_EAD_and_cost_per_trigger'


In [ ]:
dfs = [geopandas.read_parquet(fname) for fname in splits_dir.glob("*.geoparquet")]

splits = pandas.concat(dfs)

In [ ]:
splits.to_file("split_EAD_and_cost_per_trigger.gpkg", driver="GPKG", engine="pyogrio")

In [ ]:
splits.columns

In [ ]:
with rasterio.open(base_dir/"input/hazard-landslide-arup/africa-latest/ls_eq_tiled.tif") as src:
    pass

src.width, src.height, src.transform

In [ ]:
raster_height = src.height
raster_width = src.width
raster_transform = src.transform
data_col = 'hazard-_landslide_sum__road_damage_fraction_EAD'

data = numpy.zeros((raster_height, raster_width))

splits_df = splits

def get_cell_indices_kw(geom, raster_height=0, raster_width=0, raster_transform=None):
    return get_cell_indices(geom, raster_height, raster_width, raster_transform)

cell_indices_of_split_geometry = partial(get_cell_indices_kw, raster_height=src.height, raster_width=src.width, raster_transform=list(src.transform))

splits_df = pandas.concat(
    [
        splits_df[[data_col]],
        splits_df.geometry.progress_apply(cell_indices_of_split_geometry)
    ],
    axis="columns"
).rename(columns={"geometry":"cell_index"})

In [ ]:
value_per_cell = splits_df[['cell_index', data_col]].groupby('cell_index').sum()

In [ ]:
for _, item in value_per_cell.reset_index().iterrows():
    col, row = item.cell_index
    data[int(row), int(col)] = item[data_col]


In [ ]:
with rasterio.open(
        'africa-latest_damage_EAD.tif',
        'w',
        driver='GTiff',
        height=data.shape[0],
        width=data.shape[1],
        count=1,
        dtype=data.dtype,
        crs='+proj=latlong',
        transform=raster_transform,
        compress='lzw'
    ) as dataset:
    dataset.write(data, 1)